<hr>
# 第4讲-网络爬虫存储

1. 关系数据库存储-MySQL为例
2. 爬虫配合MySQL存储
3. 分布式数据存储-NoSQL数据库
4. 爬虫配合mongoDB存储
5. HDFS简介
6. 爬虫爬取入库案例

- 引子

![](./dataTm/work_pic/joke.png)

## 1.关系数据库存储-MySQL为例

### 1.1. 关系数据库介绍

假设你已经（或者有能力）获取大量数据（通过爬取），那么选择何种方式去存储数据非常重要

一般而言就爬虫问题，我们可以选择：

1. 文本文件的形式保存（比如csv）
2. 数据库
3. 文件系统

第一种

- 优势：方便，随时使用，不需要第三方的支持
- 劣势：健壮性差，扩展性差

第二种

- 优势：良好的扩展性，使用广泛
- 劣势：（硬要说的话）需要第三方支持，需要进行选择，对技术有一定要求

第三种

- 更自由，但技术要求会更高


当你选择使用**数据库**作为你的爬虫的后端存储方案时，核心的问题是：
> 哪种数据库或者组合能够最好解决你的问题？

1. 数据库类型

  - 关系型，键值型，文档型等等

2. 驱动力
  
  - 实际解决的问题场景
  - 关系型：数据库查询的灵活性 > 灵活的模式
  - 面向列的数据库：适合存储多机的海量数据
  
3. 数据库的特性

  - 除了CRUD
  - 数据库是否有模式
  - 数据库是否支持快速索引查找
  - 数据库是否支持自由定义的查询
  - 数据库是否在查询前必须先进行规划

4. 数据库的性能

  - 是否方便进行优化
  - 是否容易对读写进行优化
  - 支不支持分片，复制

5. 数据库的伸缩性

  - 横向扩展（MongoDB）
  - 纵向扩展（PostgreSQL）
  

说说爬虫：

- 一般的数据，基本上都没有问题
- 数据变大，数据量和并发都很大的时候

    - 关系型数据库容量和读写能力可能会有问题
    - 爬虫的信息（一般来说比如文本），一般不需要建立关系
    - 爬虫字段经常变化，相对严格模式的关系数据库，NoSQL会更有优势
    - 文档型（MongoDB）更加自由
    - 容易横向扩展、分片、复制
    
- 爬虫数据比较脏乱
- 有时无法预期数据的字段（先爬下来再说）
- NoSQL可以做Map Reduce优化

关系数据库：

- 以集合理论为基础的系统，实现为具有行和列的二维表
- 利用SQL（结构化查询语言）编写查询，与数据库管理系统交互
- SQLite, MySQL, PostgreSQL

### 1.2. MySQL安装

1. 安装MySQL Community Server（或者安装 AppServ）
2. 配置路径（第一次安装尽量不要进行配置的更改，设置path，记住root密码）
3. 安装Valentina Studio 7（或者MySQL Workbench，Navicat等）客户端工具
4. 测试（demo）

![](./dataTm/work_pic/mysql_start.png)

终端操作：C:\>mysql -u root –p

### 1.3. 基础SQL语句

In [ ]:
建立新用户：

CREATE USER '用户名'@'localhost' IDENTIFIED BY '密码';

In [ ]:
赋权限：

GRANT privileges ON DBname.tablename TO '用户名'@'localhost'; 
GRANT ALL ON *.* TO '用户名'@'localhost';

In [ ]:
改密码：

SET PASSWORD FOR '用户名'@'localhost' = PASSWORD('新密码');

In [ ]:
建立数据库：
mysql> CREATE DATABASE testdb1；

In [ ]:
显示数据库：
SHOW DATABASES;

In [ ]:
利用Valentina Studio连接数据库
MySQL默认端口：3306

![](./dataTm/work_pic/mysql_conn.png)

In [ ]:
使用testbd1数据库，查看其中的表格
USE testdb1;
SHOW TABLES;

In [ ]:
建立表格，查看表格：
-- 主键约束，唯一性约束

CREATE TABLE countries (
  country_code CHAR(2) PRIMARY KEY,
  country_name varchar(128) UNIQUE
);

show tables;

In [ ]:
插入表格一部分数据：

INSERT INTO countries (country_code, country_name)
VALUES ('us', 'United States'), ('mx','Mexico'), 
('au','Australia'), ('gb','United Kingdom'), 
('de','Germany'), ('ll','Loompaland');

select * from countries;

In [ ]:
唯一性约束的限制---报错

INSERT INTO countries (country_code, country_name)
VALUES ('us', 'United States');

In [ ]:
删除指定的行：

DELETE FROM countries WHERE country_code = 'll';

In [ ]:
建立新的表格：
-- 外键约束，联合主键，其它约束

CREATE TABLE cities(
    name  varchar(128) NOT NULL,
    postal_code VARCHAR(9) CHECK (postal_code <> ''),
    country_code CHAR(2) REFERENCES countries,
    PRIMARY KEY (country_code, postal_code)
);

In [ ]:
插入数据：

INSERT INTO cities
VALUES ('Portland', '87200', 'us');

In [ ]:
更新数据：

UPDATE cities
SET postal_code = '97205'
WHERE name = 'Portland';

In [ ]:
联表查询：
--inner join


SELECT cities.*, country_name
FROM cities INNER JOIN countries
ON cities.country_code = countries.country_code;

In [ ]:
建立新表（为检测复合联接查询）

CREATE TABLE venues(
    venue_id  int AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    street_address varchar(16),
    typev CHAR(8),
    postal_code VARCHAR(9),
    country_code CHAR(10),
    FOREIGN KEY (country_code, postal_code) 
    REFERENCES cities (country_code, postal_code)  MATCH FULL
);

DROP TABLE IF EXISTS venues;

SELECT * FROM venues;

In [ ]:
插入数据
进行复合联接

INSERT INTO venues (name, postal_code, country_code)
VALUES ('crystal ballroom', '97205', 'us');

SELECT v.venue_id, v.name, c.name
FROM venues v INNER JOIN cities c
ON v.postal_code=c.postal_code AND v.country_code = c.country_code;

-------

## 2.爬虫配合MySQL存储

### 2.1Python配合MySQL

MySQL和Python的连接库：

- Python2.x: MySQLdb
- Python3.x: 
  - MySQL Connector/Python(mysql.connector)
  - oursql
  - pymysql

利用conda安装

- conda search mysql
  - mysql-connector-python
  - mysql-python     :2.x
  - pymysql
  
- conda install mysql-connector-python
- **conda install pymysql**

测试：

先连接数据库

In [ ]:
# 模板
import pymysql
# 创建和数据库server的连接
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='password', db='mysql')
# connect(charset='utf8')
# cur是一种能从包括多条数据记录的结果中每次提取一条记录的机制
# 可以说认为cur就是指针
# cur一种临时的数据库对象
cur = conn.cursor() 
#cur.execute(“SQL代码”)
#cur.execute("CREATE DATABASE shdata") 
#cur.execute("USE shdata")
cur.close()
conn.close()

In [12]:
cur.close()
conn.close()

In [5]:
# 测试
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port=3306, user = 'root', passwd = '12345678', db='mysql') 
cur = conn.cursor()

try:
    cur.execute("DROP DATABASE IF EXISTS shdatabase") 
except Exception as e:
    print(e) 
finally:
    pass

In [7]:
#创建数据库 
cur.execute("CREATE DATABASE shdatabase1") 
cur.execute("USE sddatabase")

#创建表
cur.execute("CREATE TABLE users1 (id INT, name VARCHAR(16))")


0

In [10]:
cur.execute("USE shdatabase1")
cur.execute("CREATE TABLE users2 (id INT, name VARCHAR(16))")

0

In [1]:
#插入数据 
cur.execute("INSERT INTO users2 VALUES(1, ‘alice'),(2, ‘bob'),(3, ‘jack'),(4, ‘rose')") 
#'latin-1' codec can't encode character '\u2018' in position 28: ordinal not in range(256)

In [13]:
# 先建立cur，再执行字符设置
import pymysql
conn = pymysql.connect(host = '127.0.0.1', port=3306, user = 'root', passwd = '12345678', db='mysql') 
cur = conn.cursor()

cur.execute('SET NAMES utf8') 
cur.execute('SET CHARACTER SET utf8')
cur.execute('SET character_set_connection=utf8')

0

In [14]:
# 使用sddatabase数据库
cur.execute("USE shdatabase1")

0

In [15]:
#插入数据 （编码解决）
#请查看commit的作用
cur.execute("INSERT INTO users2 VALUES(1, 'alice'),(2, 'bob'),(3, 'jack'),(4, 'rose')")

4

In [16]:
#查询 
cur.execute("SELECT * FROM users2") 
for row in cur.fetchall():
    print('%s\t%s'  % row)

1	alice
2	bob
3	jack
4	rose


In [17]:
#关闭 
cur.close()
#表修改或插入数据后commit 
conn.commit() 
conn.close()

---------

### 2.2Python爬虫配合MySQL

- 继续小说例子

In [18]:
import requests
import lxml
from bs4 import BeautifulSoup
import os
import pymysql

In [19]:
def open_url(url):
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"}
    html = requests.get(url, headers=headers)
    html.encoding = 'utf-8'
    html = html.text
    return html

In [21]:
conn = pymysql.connect(host='127.0.0.1', 
                     port=3306, user='root', passwd='12345678',db='mysql',
                       use_unicode=True, charset='utf8')

In [22]:
cur = conn.cursor()

In [23]:
cur.execute('SET NAMES utf8') 
cur.execute('SET CHARACTER SET utf8')
cur.execute('SET character_set_connection=utf8')

0

In [24]:
cur.execute("USE shdatabase1")

0

In [25]:
cur.execute('create table wuxia(title varchar(128) ,author varchar(128), href varchar(64))')

0

In [26]:
html = open_url('http://wuxia.net.cn/book.html')
Soup = BeautifulSoup(html, 'lxml')
all_a = Soup.select('td p')
all_author = Soup.select('td p span')

In [27]:
for i in all_a:
        title = i.find('a').get_text()
        href = i.find('a')['href']
        author = i.find('span').get_text()
        cur.execute('insert into wuxia values(%s,%s,%s)',[title,author,href])
        #cur.execute('insert into wuxia values(%s,%s)',[title,href])
        #cur.execute('insert into wuxia values(%s,%s,%s)',[title,href,href])
        #print(title, href, author)

In [28]:
conn.commit()
#以下两步把游标与数据库连接都关闭，这也是必须的！
cur.close()
conn.close()

- 整体

In [ ]:
import requests
import lxml
from bs4 import BeautifulSoup
import os
import pymysql


def open_url(url):
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"}
    html = requests.get(url, headers=headers)
    html.encoding = 'utf-8'
    html = html.text
    return html


html = open_url(url)
    Soup = BeautifulSoup(html, 'lxml')
    all_a = Soup.select('td p')
    all_author = Soup.select('td p span')


conn = pymysql.connect(host='127.0.0.1', 
                     port=3306, user='root', passwd='123456',db='mysql',
                       use_unicode=True, charset='utf8')

cur = conn.cursor()

cur.execute('SET NAMES utf8') 
cur.execute('SET CHARACTER SET utf8')
cur.execute('SET character_set_connection=utf8')

cur.execute("USE SHdatabase")

cur.execute('create table wuxia(title varchar(128) ,author varchar(128), href varchar(64))')

for i in all_a:
        title = i.find('a').get_text()
        href = i.find('a')['href']
        author = i.find('span').get_text()
        cur.execute('insert into wuxia values(%s,%s,%s)',[title,author,href])
        #print(title, href)

conn.commit()
#以下两步把游标与数据库连接都关闭
cur.close()
conn.close()


-------

------------

## 3.分布式数据存储-NoSQL数据库

### 3.1分布式数据存储-NoSQL数据库介绍

- SQL：

  - 模型是关系型
  - 数据被存放在表中
  - 适用于每条记录都是相同类型并具有相同属性的情况
  - 存储规范需要预定义结构（模式schema）
  - 添加新的属性意味着改变整体架构
  - ACID事务支持

- NoSQL
  
  - 模型非关系型
  - 可以存储JSON、键值对等(取决于NoSQL数据库类型)
  - 并不是每条记录都要有相同的结构
  - 添加带有新属性的数据时，不会影响其他
  - 支持ACID事务，根据使用的NoSQL的数据库而有所不同
  - 一致性可以改变
  - 横向扩展能力

MongoDB是一个面向文档的，开源数据库程序，它平台无关。

MongoDB使用JSON结构的文档存储数据。数据非常灵活，不需要的Schema，但是对操作提出了较高的要求

--------

- 重要特性：

  - 支持多种标准查询类型，比如匹配(==), 大小比较(<, >),或者正则表达式
  - 可以存储几乎任何类型的数据，无论结构化，部分结构化，甚至是多态
  - 对于扩展和处理更多查询，只需添加更多的机器（横向扩展）
  - 它是高度灵活和敏捷，让您能够快速开发应用程序
  - 可以在单个文档中存储有关模型的所有信息（document based NoSQL database)
  - 可以随时更改数据库的Schema
  - 许多关系型数据库的功能也可以在MongoDB使用（如索引）
  - 作为队列的存储或者实际数据的存储
  

- 运行方面MongoDB的特点：

  - 独立服务器or独立服务器集群，可以根据需要进行扩展
  - MongoDB还通过在各个分片上自动移动数据来提供负载均衡支持
  - 具有自动故障转移支持，如果主服务器Down掉，新的主服务器将自动启动并运行
  - MongoDB的监控和备份基础设施服务
  - 由于内存映射文件，节省RAM

---------

### 3.2 MongoDB安装

- tips：
- 官网下载，这里是msi
- 直接安装，然后将path放好
- C:\Program Files\MongoDB\Server\3.4\bin;（因为你要用到mongod）
- 启动MongoDB服务之前需要必须创建数据库文件的存放文件夹，否则命令不会自动创建，且不能启动
    
> e.g.

> 建立一个D:\mongodb\data\db

>然后开启mongodb的服务：

>mongod --dbpath D:\mongodb\data\db
            
            
- 开服务的时候可能会遇到
"丢失api-ms-win-crt-runtime-l1-1-0.dll的提示，可通过运行vc_redist.2015.exe的方式解决"


![](./dataTm/work_pic/mongodb_start.png)

启动mongo命令行：

> mongo newdb

> show dbs

> use yourdb


### 3.3 MongoDB使用

利用工具：Robomongo(图形化操作)

1. 建立一个连接
2. 然后进行操作

名词比较：

SQL:MongoDB:解释

database:database:数据库

table:collection:表/集合

row:document:数据行/文档

primary key:primary key	主键/MongoDB自动将_id字段设置为主键

注意:

**在mongo中创建一个collection,只需要在该collection中插入第一条记录。由于mongo没有模式，不需要预先定义模式（比如create），可以直接使用。而且，就算你建立了一个新的collection，比如mongo book，你不插入数据，这个collection其实并不存在。**

In [ ]:
# 命令行下：
> mongo newdb
> db.towns.insert({
    name: "New York",
    population: 2220000,
    last_census: ISODate("2009-07-31"),
    famous_for: ["status of liberty", "food"],
    mayor: {
    name: "Jack",
    party: "I"
    }
    })

In [ ]:
show collections

In [ ]:
db.towns.find()

![](./dataTm/work_pic/mongo_id.png)

- _id字段，类似MySQL中自增产生的数字主键
- 由时间戳、客户端机器ID、客户端进程ID和3字节的增量计数器组成
- 优势：每台计算机上的每个进程都能够处理自己的ID生成，不会与其他mongod实例发生冲突

（思考：Mongo的分布式特征）

In [ ]:
>db.help()
>db.towns.help()
母语：JavaScript
db：JavaScript对象，包含当前数据库的有关信息
db.collection名
Mongo中，命令就是JavaScript函数
>typeof db
>typeof db.towns
>typeof db.towns.insert

```javascript

function insertCity(name, population, last_census,famous_for,mayor_info){
db.towns.insert({
    name: name,
    population: population,
    last_census: ISODate(last_census),
    famous_for: famous_for,
    mayor: mayor_info
    });
}

```

In [ ]:
insertCity("Punxsutawney", 6200, '2008-31-01', ["phil the groundhog"], {name: "Jim"})
insertCity("Portland", 52800, '2007-31-01', ["beer","food"], {name: "Sam"})

In [ ]:
db.towns.find()

In [ ]:
db.towns.find({"_id": ObjectId("58f22ba4387761c02c7bcdaa")})

In [ ]:
只查name，相当于name 为true
db.towns.find({"_id": ObjectId("58f22ba4387761c02c7bcdaa")}, {name:1})

In [ ]:
不用特定的查询语言，查询必须符合JavaScript的语法规则

In [ ]:
db.towns.find(
{famous_for:'food'},
{_id:0, name:1, famous_for:1}

    )

In [ ]:
db.towns.update(
{_id:ObjectId("58f22ba4387761c02c7bcdaa")},
{$set: {"state": "OR"}}
);

In [ ]:
# 带不带Sset的区别

db.towns.update(
{_id:ObjectId("58f22ba4387761c02c7bcdaa")},
{state: "OR"}
);

In [ ]:
db.towns.remove({"_id": ObjectId("58f22ba4387761c02c7bcdaa")}, {name:0})

--------

## 4.爬虫配合mongoDB存储

### 4.1 MongoDB配合python使用

In [2]:
import pymongo

在python中使用MongoDB，需要一个驱动

MongoDB开发者发布的官方驱动程序PyMongo

安装:conda install pymongo

In [29]:
import pymongo

In [30]:
#建立连接：
from pymongo import MongoClient
client = MongoClient()
#client = MongoClient('localhost', 27017)
#client = MongoClient('mongodb://localhost:27017')

In [31]:
#访问数据库
db = client.newdb
#db = client['pymongo_test']
#不管有没有，自动建立

In [32]:
#插入文档
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun',
    'author': 'jack'
}
result = posts.insert_one(post_data)
print('插入一条记录: {0}'.format(result.inserted_id))
# ServerSelectionTimeoutError: localhost:27017: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

插入一条记录: 58fc6d4437658e1f7cfe1c2e


In [53]:
#插入文档
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun',
    'author': 'jack'
}
result = posts.insert_one(post_data)
print('插入一条记录: {0}'.format(result.inserted_id))

插入一条记录: 58f46ab8da90b0069c05eb66


In [33]:
#同时插入多组
post_1 = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun',
    'author': 'jack'
}
post_2 = {
    'title': 'Virtual Environments',
    'content': 'Use virtual environments',
    'author': 'alice'
}
post_3 = {
    'title': 'Learning Python',
    'content': 'Learn Python, 从入门到放弃',
    'author': 'alice'
}
new_result = posts.insert_many([post_1, post_2, post_3])
print('插入多条: {0}'.format(new_result.inserted_ids))

插入多条: [ObjectId('58fc6d6d37658e1f7cfe1c2f'), ObjectId('58fc6d6d37658e1f7cfe1c30'), ObjectId('58fc6d6d37658e1f7cfe1c31')]


In [55]:
#查找
bills_post = posts.find_one({'author': 'jack'})
print(bills_post)

{'content': 'PyMongo is fun', 'title': 'Python and MongoDB', '_id': ObjectId('58f46ab8da90b0069c05eb66'), 'author': 'jack'}


In [57]:
#查找多条
alice_posts = posts.find({'author': 'alice'})
print(alice_posts)

In [58]:
#查找多条
for post in alice_posts:
    print(post)

{'content': 'Use virtual environments', 'title': 'Virtual Environments', '_id': ObjectId('58f46b02da90b0069c05eb68'), 'author': 'alice'}
{'content': 'Use virtual environments', 'title': 'Virtual Environments', '_id': ObjectId('58f46b22da90b0069c05eb6b'), 'author': 'alice'}
{'content': 'Learn Python, 从入门到放弃', 'title': 'Learning Python', '_id': ObjectId('58f46b22da90b0069c05eb6c'), 'author': 'alice'}


- ref:

- https://docs.mongodb.com/manual/core/document/

- https://realpython.com/blog/python/introduction-to-mongodb-and-python/

### 4.2 MongoDB配合python爬虫使用

<hr>

- 案例分析（I-1）

In [34]:
from bs4 import BeautifulSoup
import re
import requests
from lxml import etree
import pymongo

def getpages(url, total):
    currpage = int(re.search('(\d+)', url, re.S).group(0))
    urls = []
    for i in range(currpage, total + 1):
        link = re.sub('(\d+)', '%s' % i, url, re.S)
        urls.append(link)
    return urls

In [35]:
def saveinfo(book_name, book_author):
    connection = pymongo.MongoClient()
    BookDB = connection.BookDB
    BookTable = BookDB.books
    
    length = len(book_name)
    
    for i in range(0, length):
        books = {}
        books['name'] = str(book_name[i]).replace('\n','')
        books['author'] = str(book_author[i]).replace('\n','')
        BookTable.insert_one(books)

In [36]:
def spider(url, total):
    urls = getpages(url, total)
    for url in urls:
        html = requests.get(url)
        selector = etree.HTML(html.text)
        book_name = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/a/text()')
        book_author = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/div/a/text()')
        saveinfo(book_name, book_author)

In [128]:
#url = "http://readfree.me/psyche/?page=1"
#spider(url, 5)

- 案例分析（I-2）

In [135]:
def spider(url, total):
    urls = getpages(url, total)
    for url in urls:
        html = requests.get(url)
        bs = BeautifulSoup(html.text, "lxml")
        bookauthors = bs.select('div[class="book-author"]')
        book_author = []
        for bookauthor in bookauthors:
            a = bookauthor.getText()
            if (len(a) == 2):
                a = 'xxx'
            book_author.append(a)
            
        booknames = bs.select('div[class="book-info"]')
        book_name = []
        for bookname in booknames:
            a = bookname.getText()
            if (len(a) == 2):
                a = 'xxx'
            book_name.append(a)
    
        saveinfo(book_name, book_author)

In [136]:
url = "http://readfree.me/psyche/?page=1"
spider(url, 5)

In [132]:
# 发现错误
url = 'http://readfree.me/psyche/?page=1'
html = requests.get(url)
selector = etree.HTML(html.text)
book_name = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/a/text()')
book_author = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/div/a/text()')
print(len(book_name))
print(len(book_author))

20
19


In [134]:
# 检测错误
url = 'http://readfree.me/psyche/?page=1'
html = requests.get(url)
bs = BeautifulSoup(html.text, "lxml")
booknames = bs.select('div[class="book-author"]')
#print(booknames)
#book_name = []
for bookname in booknames:
    pass
    #print(bookname.getText())
    #print(len(bookname.getText()))

- 案例分析（I-3）

In [ ]:
# 解决方案
import re
from lxml import etree
import requests

url = 'http://readfree.me/psyche/?page=1'
html = requests.get(url)
selector = etree.HTML(html.text)
book_name = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/a/text()')
book_author = re.findall('23">(.*?)</a>', html.text)
#book_author = selector.xpath('//*[@id="container"]/ul/li//div/div[2]/div/a/text()')
#names = re.findall('alt=(.*?)/>', html.text)
#names = re.findall('<a class="pjax" href="/book/\d*?/">([\s\S]*?)</a>', html.text)
print(len(book_name))
print(len(book_author))

tips:

在xpath或者css 选择器不能很好的进行信息抽取时，可以利用正则表达式

而且往往正则表达式能够给出最好的方案和性能，只是进阶有一些学习成本

--------------------

- 案例分析（II）

针对Ajax异步加载，发现json数据，可以直接入库

In [37]:
import requests

In [38]:
url = 'http://www.guokr.com/scientific/'
# 和花瓣的区别，不用找link，直接把json数据入库

In [39]:
import json
url = 'http://www.guokr.com/apis/minisite/article.json?retrieve_type=by_subject&limit=20&offset=78&_=1492275395425'
html = requests.get(url)
data_list = json.loads(html.text)
print(data_list.keys())
for data in data_list['result']:
    #pass
    print(data)

dict_keys(['total', 'limit', 'now', 'offset', 'ok', 'result'])
{'minisite': {'date_created': '2010-10-20T16:20:44+08:00', 'icon': 'http://3.im.guokr.com/5xjuxoJWksTNiljSjKy-q7mKJDn3DYm7aHZetMDKj-BuAAAAWgAAAEpQ.jpg', 'key': 'microsf', 'introduction': '通过科学抵达的魔法世界，带你领略科学幻想维度', 'url': 'http://www.guokr.com/site/microsf/', 'name': '微科幻'}, 'authors': [{'gender': 'male', 'master_category': 'personal', 'is_exists': True, 'amended_reliability': '0', 'url': 'http://www.guokr.com/i/1248922694/', 'is_title_authorized': True, 'avatar': {'small': 'http://1.im.guokr.com/2rLrV81nV3ZMOJK93aPNhPJWoVHWk_84XlBl7LL3-vugAAAAoAAAAEpQ.jpg?imageView2/1/w/24/h/24', 'large': 'http://1.im.guokr.com/2rLrV81nV3ZMOJK93aPNhPJWoVHWk_84XlBl7LL3-vugAAAAoAAAAEpQ.jpg?imageView2/1/w/160/h/160', 'normal': 'http://1.im.guokr.com/2rLrV81nV3ZMOJK93aPNhPJWoVHWk_84XlBl7LL3-vugAAAAoAAAAEpQ.jpg?imageView2/1/w/48/h/48'}, 'nickname': 'Articdoctor', 'followers_count': 1121, 'resource_url': 'http://apis.guokr.com/community/user/knkqx

In [40]:
from bs4 import BeautifulSoup
import requests
import json
import pymongo

url = 'http://www.guokr.com/scientific/'

def getData(url):
    connection = pymongo.MongoClient('localhost', 27017)
    guokeDB = connection.guokeDB
    guokeData = guokeDB.guokeData
    html = requests.get(url)
    data_list = json.loads(html.text)
    print(data_list.keys())
    for data in data_list['result']:
        guokeData.insert_one(data)

urls = ['http://www.guokr.com/apis/minisite/article.json?retrieve_type=by_subject&limit=20&offset={}&_=1462252453410'.format(str(i)) for i in range(20, 100, 20)]
for url in urls:
    getData(url)

dict_keys(['total', 'limit', 'now', 'offset', 'ok', 'result'])
dict_keys(['total', 'limit', 'now', 'offset', 'ok', 'result'])
dict_keys(['total', 'limit', 'now', 'offset', 'ok', 'result'])
dict_keys(['total', 'limit', 'now', 'offset', 'ok', 'result'])


------------

<hr>

## 5.HDFS简介

HDFS定义：

- HDFS:Hadoop Distributed File System
- Hadoop项目的核心子项目，是Hadoop主要的一个分布式文件系统
- 比较流行的是将抓取的网页保存在分布式文件系统上

Hadoop是一个由Apache基金会所开发的分布式系统基础架构。

- 支持用户在不了解分布式底层细节的情况下，开发分布式程序
- 充分利用集群的威力进行高速运算和存储。
- Hadoop实现了一个分布式文件系统（Hadoop Distributed File System）-HDFS
- HDFS有高容错性的特点，并且设计用来部署在低廉的（low-cost）硬件上
- 提供高吞吐量（high throughput）来访问应用程序的数据
- 适合大数据集的应用。
- Hadoop的框架最核心的设计就是：HDFS和MapReduce
- HDFS为海量的数据提供了存储，则MapReduce为海量的数据提供了计算

爬虫或者类似应用的做法：

- 将HDFS看成数据库（虽然不是）。做到比NoSQL更彻底
- 从结构化的关系数据库，到无模式的NoSQL，到HDFS文件系统（更灵活，更强大）
- **主要考虑到后续操作**

<hr>
Hadoop作为大数据离线分析的一套框架，通常会和HDFS文件系统组合，进行数据的分析：

- 注意：我们的爬虫爬下来网页只是第一步，后续的分析才是重头戏
- 如果面对巨大的数据量，会利用Hadoop这样分布式系统框架进行分析，那么从数据源（各个网站的页面）到HDFS成为了一个一步到位的方式。

![](./dataTm/work_pic/hdfsread.png)

![](./dataTm/work_pic/hdfswrite.png)

基本操作与类Unix操作系统类似：

```code
$ bin/hadoop dfs -ls
$ bin/hadoop dfs -ls doc_name
$ bin/hadoop dfs -put local_file hdfs_file
# 理解成上传至数据库
$ bin/hadoop dfs -get hdfs_file local_file
# 理解成从数据库获取至本地文件
$ bin/hadoop dfs -cat hdfs_doc/*



```

优点：

1. 一个文件可以大于每个磁盘
2. 文件不用全在一个磁盘上
3. 简化了存储子系统的设计
4. HDFS是基于主从结构（master/slaver）构件（对比爬虫）

Hadoop基于Java？
1. Python

![](./dataTm/work_pic/pyhdfs.png)

In [ ]:
import pyhdfs

fs = pyhdfs.connect("192.168.1.1", 9000)
pyhdfs.get(fs, "/path/to/remote-src-file", "/path/to/local-dst-file")

f = pyhdfs.open(fs, "/user/project/quotes.txt", "w")
pyhdfs.write(fs, f, "hello hadoop and hello big data.")
pyhdfs.close(fs, f)

pyhdfs.disconnect(fs)

一种新思路：

Docker介绍

- Docker 是 PaaS 提供商 dotCloud 开源的一个基于 LXC 的高级容器引擎，源代码托管在 Github 上, 基于go语言并遵从Apache2.0协议开源。

- ETCD是用于共享配置和服务发现的分布式，一致性的**KV存储系统**。该项目目前最新稳定版本为2.3.0.ETCD是CoreOS公司发起的一个开源项目，授权协议为Apache。

- zerg基于Docker的分布式爬虫服务
- https://github.com/huichen/zerg

![](./dataTm/work_pic/zerg.png)

特性

- 多机多 IP，充分利用 IP 资源
- 服务自动发现和注册（基于 etcd 和 registrator）
- 负载均衡
- 服务端客户端通信基于 gRPC，支持多种编程语言的客户端
- 可设置抓取超时
- 支持 GET、HEAD、POST 方法
- 支持自定义 header

## 6.爬虫爬取入库案例

In [60]:
# 见mysql与mongdb部分

tips:

附：利用AppServ安装web server包括MySQL的注意点

![](./dataTm/work_pic/VC11.png)

![](./dataTm/work_pic/vc2012.png)

![](./dataTm/work_pic/vc2015.png)

![](./dataTm/work_pic/vc2015suc.png)

![](./dataTm/work_pic/appserv.png)

![](./dataTm/work_pic/mysql_setup.png)

![](./dataTm/work_pic/start_mysql_apache.png)